# Compute Sponsorship Centrality Metrics

This notebook computes simple sponsorship centrality-style metrics on the sample dataset.

This analysis treats:
- `total_sponsors` ~ in-degree centrality in the sponsorship graph
- `total_sponsoring` ~ out-degree centrality

It reports normalized scores and a composite "influence" score that can be used to identify key actors in the ecosystem for case studies.

In [ ]:
import pandas as pd
from data_loader import load_sample_data

In [ ]:
def compute_centrality(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    for col in ("total_sponsors", "total_sponsoring"):
        if col not in df.columns:
            df[col] = 0

    max_in = df["total_sponsors"].max() or 1
    max_out = df["total_sponsoring"].max() or 1

    df["in_degree_centrality"] = df["total_sponsors"] / max_in
    df["out_degree_centrality"] = df["total_sponsoring"] / max_out

    # Simple composite: weight incoming sponsorships more heavily, since
    # they capture prominence and perceived value.
    df["influence_score"] = 0.7 * df["in_degree_centrality"] + 0.3 * df["out_degree_centrality"]

    return df

## Load Data and Compute Centrality

In [ ]:
df = load_sample_data()
df_c = compute_centrality(df)

## Display Top Actors by Influence Score

In [ ]:
cols_to_show = [
    "username",
    "name",
    "type",
    "total_sponsors",
    "total_sponsoring",
    "in_degree_centrality",
    "out_degree_centrality",
    "influence_score",
]
cols_to_show = [c for c in cols_to_show if c in df_c.columns]

top_n = 25
ranked = df_c.sort_values("influence_score", ascending=False)[cols_to_show].head(top_n)

pd.set_option("display.max_columns", 20)
print("\nTop actors by influence_score (based on sponsorship in/out-degree):\n")
ranked